In [1]:
'''
将产成品仓库的存货编码转为销售部的存货编码
'''

import pandas as pd
import numpy as np
import re
import openpyxl
import easygui
import os


#仓库销售编码字典
fname_dic =  r"F:\a00nutstore\008\zw08\用友报价\仓库销售编码字典.xlsx"
df_dic = pd.read_excel(fname_dic)
dic_ck_xs = dict(zip(df_dic['仓库货号'],df_dic['销售货号']))
dic_xs_ck = dict(zip(df_dic['销售货号'],df_dic['仓库货号']))







In [2]:
fname_xs = r"F:\a00nutstore\008\zw08\用友报价\销售仓库价格库存1.xlsx"
# 销售编码
def xiaoshou(fname_xs):
    df_xs = pd.read_excel(fname_xs)
    df_xs = df_xs[~df_xs['类别'].isna()]
    return df_xs
df_xs = xiaoshou(fname_xs)

xs_lst = [
    '类别',
    '货号',
    '品名',
    '销售部存货大类编码',
    '含量',
    '汉办',
    '汉口北电商',
    '外地',
    '规格型号',
    '存货代码',
    '仓库货号'
]
df_xs = df_xs[xs_lst]


In [3]:
#产成品仓库库存
def chuliChangku(fname,sheet_name):
    df = pd.read_excel(fname,sheet_name = sheet_name)
    #删除第一列的空行
    df = df[~df['类别'].isna()]
    df = df[~df['类别'].str.contains('小计|累计|合   计')]
    df = df[~(abs(df['本日数量'] - 0) < 0.00001)]
    return df
fname_canchengpin = r"F:\a00nutstore\008\zw08\用友报价\产成品库存\成品仓库数据7.25.xls"
sheet_name  =  0
df_canchengpin = chuliChangku(fname_canchengpin,sheet_name)

def addXiaoshouhuohao(df,dic_ck_xs):
    df_ck = df.assign(销售货号 = df['货号'].map(dic_ck_xs))
    # gp = df_ck.groupby('货号')
    gp = df_ck.groupby('货号')      #按销售货㞻汇总，使数据唯一性，为下一步合并做好准备
    df_ck = pd.DataFrame(gp.sum())
    df_ck = df_ck[['销售货号','本日数量']]
    df_ck = df_ck.reset_index()
    return df_ck
df_ck0 = addXiaoshouhuohao(df_canchengpin,dic_ck_xs)  #仓库
df_ck = df_ck0[['本日数量','销售货号','货号']]
df_ck = df_ck.rename(columns = {'货号':'仓库货号'})
df_ck_newAdd = df_ck.loc[df_ck['销售货号'].isin([0,'0'])]
df_ck['销售货号'] = np.where(df_ck['销售货号'].isin([0,'0']),df_ck['仓库货号'],df_ck['销售货号'] )
df_ck.to_excel('仓库库存.xlsx',index = False)
df_ck_newAdd = df_ck_newAdd[['仓库货号','销售货号']]
df_dic = pd.concat([df_dic,df_ck_newAdd])
df_dic.to_excel(fname_dic,index = False)

df_ck_newAdd.to_excel('仓库库存新增.xlsx',index = False)






In [4]:
def weiyihuXiaoshouhuohao(df_xs, df_ck, dic_xs_ck):
    merge = pd.merge(df_xs, df_ck, left_on='货号', right_on='销售货号', how='right')
    merge['本日数量本'] = merge['本日数量']
    merge['本日数量件'] = merge['本日数量'] / merge['含量']
    del merge['本日数量']
    del merge['销售货号']
    gp = merge.groupby('货号')
    dic_agg = {'含量': 'mean', '汉办': 'mean', '汉口北电商': 'mean', '外地': 'mean', '本日数量本': 'sum'}
    right = gp.agg(dic_agg)
    right = right.reset_index()
    # right = right.rename(columns = {"销售货号":'货号'})
    left = df_xs[[
        '类别',
        '货号',
        '品名',
        '销售部存货大类编码',
         '规格型号',
    '存货代码'
        
    ]]
    mg = pd.merge(left, right, left_on='货号', right_on='货号',how = 'right')
    mg['本日数量件'] = mg['本日数量本'] / mg['含量']
    mg = mg.assign(仓库货号=mg['货号'].map(dic_xs_ck))
    return mg


result_beforeNew = weiyihuXiaoshouhuohao(df_xs, df_ck, dic_xs_ck)
result_beforeNew.to_excel('销售仓库价格库存0725.xlsx',index = False)

In [5]:
#7月16-7月25日新系统累计出库
lst = [
'仓库编码',
    '仓库',   
    '存货分类 (1级)',
 '存货分类 (2级)',
 '存货分类 (3级)',
 '存货分类 (4级)',
 '存货分类 (5级)',
 '存货编码',
 '存货',
 '存货代码',
  '数量(主单位)',
 '平均单价',
 '金额',
 '数量(辅单位)',
 '计量单位组合',
 '数量(主单位).1',
 '平均单价.1',
 '金额.1',
 '数量(辅单位).1',
 '计量单位组合.1',
 '数量(主单位).2',
 '平均单价.2',
 '金额.2',
 '数量(辅单位).2',
 '计量单位组合.2',
 '数量(主单位).3',
 '平均单价.3',
 '金额.3',
 '辅数量',
 '计量单位组合.3']
lst2 = [
    '仓库编码',
    '仓库',  
'class01',
 'class02',
 'class03',
 'class04',
 'class05',
 'code',
 'stock',
 'content',
  'begin_ben',
 '平均单价',
 '金额',
 'begin_jian',
 '计量单位组合',
 'ruku_ben',
 '平均单价.1',
 '金额.1',
 'ruku_jian',
 '计量单位组合.1',
 'chuku_ben',
 '平均单价.2',
 '金额.2',
 'chuku_jian',
 '计量单位组合.2',
 'end_ben',
 '平均单价.3',
 '金额.3',
 'end_jian',
 '计量单位组合.3']

lst3 =  [
 'class02',
 'class05',
 'code',
 'stock',
 'content',
 'begin_jian',
 'ruku_jian',
    'chuku_ben',
 'chuku_jian',
 'end_jian',
]




def chuli(fname):
    df =  pd.read_excel(fname,skiprows = 7)
    df = df.iloc[:,1:]
    df.columns = lst
    df.columns = lst2
    df['content'] = df['end_ben']/df['end_jian']
    df = df[df['class01'] != '制表人:']
    df = df[df['class01'] != '合计：']
    df = df[df['class01'].notnull()]
    df = df[lst3]
    return df

fname = r"F:\a00nutstore\008\zw08\用友报价\产成品库存\收发存汇总表0716-0725 -001.xlsx"
chuku = chuli(fname)

chuku_new = chuku[['code','begin_jian','ruku_jian','chuku_ben','chuku_jian']]
chuku_new = chuku_new.groupby('code').sum()
chuku_new = chuku_new.reset_index()
chuku_new

,code,begin_jian,ruku_jian,chuku_ben,chuku_jian
0,1101,37.0,0.02,1.0,0.02
1,1103,15.0,0.10,6.0,0.10
2,1105,15.0,0.00,0.0,0.00
3,1106,118.0,0.00,120.0,2.00
4,1107,19.0,0.00,0.0,0.00
...,...,...,...,...,...
1540,瑞典A460g线条,56.0,0.00,0.0,0.00
1541,瑞典A470g线条,1.0,0.00,0.0,0.00
1542,美国A6铁订本(枫叶)*,9.0,0.00,0.0,0.00
1543,美国A6铁订本*,7.0,0.00,0.0,0.00


In [ ]:
chuku_new.to_excel('测试期间出库.xlsx',index = False)

In [9]:
#期初本、期初件
fname0 = r"F:\a00nutstore\008\zw08\用友报价\销售部001库存20240716正.xlsx"
df0 = pd.read_excel(fname0)
df0 = df0[['货号','本日数量本','本日数量件']]
df0 = df0.rename(columns = {'本日数量本':'qichu_ben','本日数量件':'qichu_jian'})
df0

,货号,qichu_ben,qichu_jian
0,1101,2220,37.0
1,1103,900,15.0
2,1105,900,15.0
3,1106,7080,118.0
4,1107,1140,19.0
...,...,...,...
1529,瑞典A460g方格,720,18.0
1530,瑞典A460g线条,2240,56.0
1531,瑞典A470g线条,40,1.0
1532,商品卡,113400,21.0


In [10]:
result0 = pd.merge(result_beforeNew,chuku_new,left_on = '货号',right_on = 'code',how = 'left')
result1 = pd.merge(result0,df0,on = '货号',how = 'left')
result1

,类别,货号,品名,销售部存货大类编码,规格型号,存货代码,含量,汉办,汉口北电商,外地,本日数量本,本日数量件,仓库货号,code,begin_jian,ruku_jian,chuku_ben,chuku_jian,qichu_ben,qichu_jian
0,普通账芯,1101,16K总分类账,01010201,A2账芯,账本,60.0,5.5200,5.5200,5.5200,2220,37.0,1101,1101,37.0,0.02,1.0,0.02,2220.0,37.0
1,普通账芯,1103,16K银行存款日记账,01010201,A2账芯,账本,60.0,5.5200,5.5200,5.5200,900,15.0,1103,1103,15.0,0.10,6.0,0.10,900.0,15.0
2,普通账芯,1105,16K现金日记账,01010201,A2账芯,账本,60.0,5.5200,5.5200,5.5200,900,15.0,1105,1105,15.0,0.00,0.0,0.00,900.0,15.0
3,普通账芯,1106,16K三栏式明细分类账,01010201,A2账芯,账本,60.0,5.5200,5.5200,5.5200,7020,117.0,1106,1106,118.0,0.00,120.0,2.00,7080.0,118.0
4,普通账芯,1107,16K固定资产明细分类账,01010201,A2账芯,账本,60.0,5.5200,5.5200,5.5200,1140,19.0,1107,1107,19.0,0.00,0.0,0.00,1140.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,外贸订制品,瑞典A460g线条,瑞典A460g线条,0107,B14外贸部订制品（人民币,抄本,40.0,0.0000,0.0000,0.0000,2240,56.0,瑞典A460g线条,瑞典A460g线条,56.0,0.00,0.0,0.00,2240.0,56.0
1568,外贸订制品,瑞典A470g线条,瑞典A470g线条,0107,B14外贸部订制品（人民币,抄本,40.0,0.0000,0.0000,0.0000,40,1.0,瑞典A470g线条,瑞典A470g线条,1.0,0.00,0.0,0.00,40.0,1.0
1569,外贸订制品,美国A6铁订本(枫叶)*,美国A6铁订本(枫叶)*,0107,B14外贸部订制品（人民币,抄本,36.0,0.0000,0.0000,0.0000,324,9.0,美国A6铁订本*,美国A6铁订本(枫叶)*,9.0,0.00,0.0,0.00,324.0,9.0
1570,外贸订制品,美国A6铁订本*,美国A6铁订本(枫叶)*,0107,B14外贸部订制品（人民币,抄本,36.0,0.0000,0.0000,0.0000,252,7.0,美国A6铁订本(枫叶)*,美国A6铁订本*,7.0,0.00,0.0,0.00,252.0,7.0


In [11]:
result1['期末件'] = result1['本日数量件'] - result1['chuku_jian']
result1['期末本'] = result1['本日数量本'] - result1['chuku_ben']
result1['变动件'] = result1['期末件'] - result1['qichu_jian']
result1['变动本'] = result1['期末本'] - result1['qichu_ben']

In [12]:
result1.to_excel('产成品变动.xlsx',index = False)


In [ ]:
#本月新增产品，将含量
df_xsEmpty = df_xs.iloc[:0]    #留df架构
df_xsAdd = pd.concat([df_xsEmpty,df_ck_newAdd])
df_xsAdd['仓库货号']= df_xsAdd['仓库货号'].astype('str')
df_content = df_canchengpin[['类别','货号','品名','含量']]
df_content['货号']= df_content['货号'].astype('str')
df_New = pd.merge(df_xsAdd,df_content,left_on = '仓库货号',right_on = '货号',how = 'left')
df_New['含量_x'] = df_New['含量_y'] 
df_New['类别_x'] = df_New['类别_y'] 
df_New['品名_x'] = df_New['品名_y'] 
df_New

In [ ]:
cols = ['类别_x',
 '货号_x',
 '品名_x',
 '销售部存货大类编码',
 '含量_x',
 '汉办',
 '汉口北电商',
 '外地',
 '规格型号',
 '存货代码',
 '仓库货号',
  '含量_y']
df_New = df_New[cols]

del df_New['含量_y'] 
cols = cols[:-1]
cols = [x.replace('_x','') for x in cols]
df_New.columns = cols
df_New.to_excel('新增产品.xlsx',index = False)